In [ ]:
!pip install pysmt
!pysmt-install --z3
!pip install ortools
!pip install networkx

# TP01

**Exercício 1.**

Enunciado:

Pretende-se construir o horário semanal de aulas de uma turma.
  1. Existe um conjunto de salas S classificadas em “grandes” e “pequenas”.
  2. O tempo do horário está organizado em “slots” de uma hora. O total do tempo disponível é 5 horas de manhã e 5 horas às tarde.
  3. Existe um conjunto D de disciplinas.  Cada disciplina tem um atributo d com valor 1 ou 2, que classifica a duração de cada sessão (um ou dois “slots”) , um atributo a entre 2 e 3 que define o número de sessões semanais e um atributo s entre 0 e 1 que diz se a sessão necessita de uma sala grande ou não.
  4. Existe um conjunto P de professores. Cada professor tem associado um conjunto  h das            disciplinas que está habilitado a lecionar.
  5. O horário está organizado em sessões concorrentes onde cada sessão é definido por uma disciplina desce que salas e professores verifiquem as seguintes restrições
    *   Para cada disciplina todas as aulas decorrem na mesma sala e com o mesmo professor.
    *   O número total de horas lecionadas por cada professor está num intervalo de mais ou menos 20% do número médio de horas lecionadas pela totalidade dos professores.
    *   Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
    * Em cada disciplina, cada aula é lecionada por um professor habilitado para essa disciplina e ocorre numa sala de tamanho apropriado à disciplina.

    Use o package ortools para encontrar uma solução que verifique todas as restrições e maximize o número de partes de dia (manhãs ou tardes) que estão livres de qualquer aula.


* S - salas
* E - dia
* H - hora/slot
* D - cadeira/disciplina
* P - Professor

In [ ]:
from ortools.linear_solver import pywraplp

salas_g = 2 # auditorios - salas grandes
salas_p = 4 # salas pequenas
dias = {1:"segunda",2:"terça",3:"quarta",4:"quinta",5:"sexta"}
slots_manha = 5
slots_tarde = 5
professores = 4
disciplinas = 4

aulas = {1:(2,2,1), 2:(1,3,0), 3:(2,3,1), 4:(2,2,0)} # cadeira:(n de slots a ocupar, n de aulas p semana, sala grande ou pequena)
leciona = {1:[1,3], 2:[3,4], 3:[2], 4:[4]} # quais cadeiras os professores podem lecionar
salas_grandes= [1,2]
salas_pequenas = [3,4,5,6]

solver = pywraplp.Solver.CreateSolver('SCIP')

# X RELACIONA SALAS, DIAS, SLOTS E CADEIRAS
x={}

for s in range(1,salas_p +salas_g+1):
  x[s] = {}
  for d in range(1,5+1):
    x[s][d]={}
    for t in range(1,slots_manha+slots_tarde+1):
      x[s][d][t]={}
      for c in range(1,disciplinas+1):
        x[s][d][t][c]=solver.BoolVar('x[%i][%i][%i][%i]' %(s,d,t,c))

# Y RELACIONA PROF, DIA, HORA E CADEIRA
y={}
for p in range(1,professores+1):
  y[p]={}
  for d in range(1,5+1):
    y[p][d]={}
    for t in range(slots_manha + slots_tarde +1):
      y[p][d][t]={}
      for c in range(1,disciplinas+1):
        y[p][d][t][c]=solver.BoolVar('y[%i][%i][%i][%i]' %(p,d,t,c))

# Z REPRESENTA A EXISTENCIA DE AULA NO DIA
z={}
for d in range(1,5+1):
  z[d]={}
  for c in range(1,disciplinas+1):
    z[d][c]=solver.BoolVar('z[%i][%i]' %(d,c))

##############################################
# Restrições

#1 Nenhum stor pode lecionar aulas ao msm tempo
for p in range(1, professores + 1):
    for d in range(1, 5 + 1):
        for t in range(1, slots_manha + slots_tarde + 1):
            solver.Add(sum(y[p][d][t][c] for c in range(1, disciplinas + 1)) <= 1)

#2 Um professor só pode dar aula numa cadeira que esteje habilitado
for p in range(1,professores+1):
  for d in range(1,5+1):
    for t in range(1,slots_manha+slots_tarde+1):
      for c in range(1,disciplinas+1):
        if c not in leciona[p]:
          solver.Add(y[p][d][t][c]==0)

#3 Cada sala só pode ter uma aula por vez
for s in range(1, salas_g +salas_p+1):
    for d in range(1, 5 + 1):
        for t in range(1, slots_manha + slots_tarde + 1):
            solver.Add(sum(x[s][d][t][c] for c in range(1, disciplinas + 1)) <= 1)

#4 Existe um numero fixo de aulas semanais
for aula,tuplo in aulas.items():
  n_slots,n_aulas,t_sala = tuplo
  solver.Add(sum([x[s][d][t][aula] for s in range(1,salas_p + salas_g+1) for d in range(1,5+1) for t in range(1,slots_manha+slots_tarde+1)]) == n_slots*n_aulas)

#5 Uma aula deve ocorrer em uma sala grande ou pequena
for c in range(1, disciplinas + 1):
    for d in range(1, 5 + 1):
        for t in range(1, slots_manha + slots_tarde + 1):
            if aulas[c][2] == 1:  # Se a aula requer uma sala grande
                solver.Add(sum(x[s][d][t][c] for s in range(1, salas_g + 1)) == 1)
            else:  # Se a aula requer uma sala pequena
                solver.Add(sum(x[s][d][t][c] for s in range(salas_g + 1, salas_p + salas_g + 1)) == 1)

#6 Cada professor leciona dentro do intervalo de mais ou menos 20% da média
total_horas_aula = {}  # Dicionário para manter o total de horas de aula para cada professor
for p in range(1, professores + 1):
    total_horas_aula[p] = solver.Sum(y[p][d][t][c] for p in range(1, professores + 1)
                                    for d in range(1, 6)
                                    for t in range(1, slots_manha + slots_tarde + 1)
                                    for c in range(1, disciplinas + 1))

media_horas_por_semana = solver.Sum(total_horas_aula[p] for p in range(1, professores + 1)) / 5  # Média por semana
for p in range(1, professores + 1):
    solver.Add(total_horas_aula[p] >= 0.8 * media_horas_por_semana)  # Pelo menos 80% da média
    solver.Add(total_horas_aula[p] <= 1.2 * media_horas_por_semana)  # No máximo 120% da média

# Variáveis que representam os slots livres (0) e ocupados (1)
horarios_livres = {}
for d in range(1, 6):
    for t in range(1, slots_manha + slots_tarde + 1):
        horarios_livres[d, t] = solver.BoolVar(f'horario_livre[{d}][{t}]')

# Função objetivo: maximize o número de slots livres em manhãs e tardes
solver.Maximize(solver.Sum(horarios_livres[d, t] for d in range(1, 6) for t in range(1, slots_manha + slots_tarde + 1)))

# Após a resolução do modelo
if solver.Solve() == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada:")
    for d in range(1, 6):
        for t in range(1, slots_manha + slots_tarde + 1):
            for c in range(1, disciplinas + 1):
                for s in range(1, salas_p + salas_g + 1):
                    if x[s][d][t][c].solution_value() == 1:
                        print(f"Dia {dias[d]}, Slot {t}, Disciplina {c}, Sala {s}")

else:
    print("Não foi possível encontrar uma solução ótima.")


In [ ]:
from ortools.linear_solver import pywraplp

salas_g = ["a1","a2"] # auditorios - salas grandes
salas_p = ["s1","s2","s3","s4"] # salas pequenas
dias = ["segunda","terça","quarta","quinta","sexta"]
slots_manha = 5
slots_tarde = 5
professores = ["p1","p2","p3","p4"]
disciplinas = ["d1","d2","d3","d4"]

aulas = {"d1":(2,2,1), "d2":(1,3,0), "d3":(2,3,1), "d4":(2,2,0)} # quais sao os atributos das cadeiras em formato (n de slots a ocupar, n de sessões, se precisa de sala grande-1 ou não-0)

leciona = {"p1":["d1","d3"], "p2":["d3","d4"], "p3":["d2"], "p4":["d4"]} # quais cadeiras os professores podem lecionar

solver = pywraplp.Solver.CreateSolver('SCIP')

horario={}
for d in disciplinas:
  for p in professores:
    for s in salas_g + salas_p:
      for dia in dias:
        for slot in range(slots_manha + slots_tarde):
          horario[d,p,s,dia,slot]= solver.IntVar(0,1,f'x[{d},{p},{s},{dia},{slot}]')

#========================
# RESTRIÇOES
#========================

# 1. Cada cadeira tem um num fixo de sesões por semana, na msm sala e com mesmo docente
for d in disciplinas:
    for p in professores:
        for s in salas_g + salas_p:
            for dia in dias:
                for slot in range(slots_manha + slots_tarde):
                    nsessoes = aulas[d][1]
                    solver.Add(solver.Sum(horario[d, p, s, dia, slot] for s in salas_g + salas_p for dia in dias for slot in range(slots_manha + slots_tarde)) == nsessoes)


# 2. O número total de horas lecionadas por cada professor é mais ou menos 20% do número médio de horas lecionadas pela totalidade dos professores.
horasprof = sum(aulas[d][0] * aulas[d][1] for d in disciplinas) / len(professores)
for p in professores:
    horas_total_aulas = solver.Sum(aulas[d][0] * aulas[d][1] * solver.Sum(horario[d, p, s, dia, slot] for s in salas_g + salas_p for dia in dias for slot in range(slots_manha + slots_tarde)) for d in disciplinas)
    solver.Add(horas_total_aulas >= 0.8 * horasprof)
    solver.Add(horas_total_aulas <= 1.2 * horasprof)


# 3. Nenhuma sala pode ser ocupada simultaneamente por mais do que uma aula e nenhum professor pode lecionar simultaneamente mais do que uma aula.
for dia in dias:
    for slot in range(slots_manha + slots_tarde):
        for s in salas_g + salas_p:
            solver.Add(solver.Sum(horario[d, p, s, dia, slot] for d in disciplinas for p in professores) <= 1)


# 4. Restrições de disponibilidade das salas de acordo com o atributo s
for d in disciplinas:
    for dia in dias:
        for slot in range(slots_manha + slots_tarde):
            if aulas[d][2] == 1:  # Requer sala grande
                solver.Add(solver.Sum(horario[d, p, s, dia, slot] for p in professores for s in salas_p) == 0)

# 5. Restrição para evitar que uma aula de 2 slots seja dividida entre manhã e tarde
for d in disciplinas:
    for p in professores:
        for s in salas_g + salas_p:
            for dia in dias:
                for slot in range(slots_manha + slots_tarde - 1):
                    # Verifique se a aula começa de manhã e termina à tarde
                    if slot < slots_manha - 1 and slot + aulas[d][0] > slots_manha:
                        solver.Add(horario[d, p, s, dia, slot] + horario[d, p, s, dia, slot + 1] <= 1)
                    # Verifique se a aula começa à tarde e termina à tarde
                    elif slot >= slots_manha and slot + aulas[d][0] > slots_manha + slots_tarde:
                        solver.Add(horario[d, p, s, dia, slot] + horario[d, p, s, dia, slot + 1] <= 1)

problema = solver.Sum(1 - solver.Sum(horario[d, p, s, dia, slot] for d in disciplinas for p in professores for s in salas_g + salas_p for dia in dias for slot in range(slots_manha + slots_tarde)) for dia in dias for slot in range(slots_manha + slots_tarde))
solver.Maximize(problema)

# Resolva o problema
status = solver.Solve()

# Verificar o status da solução
if status == pywraplp.Solver.OPTIMAL:
    print('Solução ótima encontrada:')
    for dia in dias:
        for slot in range(slots_manha + slots_tarde):
            for p in professores:
                for d in disciplinas:
                    for s in salas_g + salas_p:
                        if horario[d, p, s, dia, slot].solution_value():
                            print(f'Disciplina {d}, Professor {p}, Sala {s}')
            print()
else:
    print('O solver não conseguiu encontrar uma solução ótima. Status:', status)
